# Introdução ao BERT - Gerando *sentence embeddings*

Neste *notebook* você verá um exemplo de como usar o [BERT](https://arxiv.org/abs/1810.04805) para extrair os *embeddings* de sentenças, além de conhecer mais sobre este modelo. 

Fontes:  

- [BramVanroy/bert-for-inference](https://github.com/BramVanroy/bert-for-inference).
- [Hugging Faces - Github](https://github.com/huggingface/transformers) e [Hugging Faces - site](https://huggingface.co/)


In [2]:
import torch
from transformers import BertModel, BertTokenizer

### O tokenizador (*tokenizer*)

Os modelos *deep learning* trabalham com tensors, que são basicamente vetores, que por sua vez são um grupo de números. Para começar, o texto de entrada (*string*) precisa ser convertido em um tipo de data que os modelos possam usar (números). Essa é a tarefa do tokenizador.

Para língua portuguesa, podemos usar a versão multilíngue do BERT, ou usar versões pré-treinadas em português brasileiro como o [BERTimbau](https://github.com/neuralmind-ai/portuguese-bert) e [BioBERTpt](https://github.com/HAILab-PUCPR/BioBERTpt), este último treinado no domínio clínico e biomédico.

In [3]:
# Inicializando o tokenizador com um modelo multilingual pré-treinado
# este modelo tem suporte a 100 idiomas, incluindo português 
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # versao BERT em inglês

# para usar um modelo BERT especifico para portugues, primeiro baixe o modelo
# fazer download do modelo e coloque em 'path/to/bert_dir'
#model = BertModel.from_pretrained('path/to/bert_dir') 

Durante o pré-treinamento, o tokenizador também é treinado, gerando um vocabulário conhecido. Cada palavra do vocabulário é atribuída a um índice (número), que pode ser usado pelo modelo. 

Para lidar com problemas das palavras que o tokenizador não conhece (*out-of-vocabulary* ou OOV), uma técnica é utilizada para garantir que o tokenizador aprenda "subpalavras". Desta forma, quando usamos os modelos pré-treinados, não teremos problemas de OOV. Quando o tokenizador não reconhece uma palavra, que não está no vocabulário, ele divide a palavra em pequenas unidades que são conhecidas. O tokenizador do BERT usa [WordPiece](https://arxiv.org/pdf/1609.08144.pdf) para dividir os tokens.

In [21]:
# Converte a string "barras de granola" para um vocabulário tokenizado 
granola_ids = tokenizer.encode('barras de granola')
# Imprime os IDs
print('granola_ids', granola_ids)
print('tipos de granola_ids', type(granola_ids))
# Converte os IDs para o item do vocabulário
# As subpalavras (sufixo) começam com "##", indicando que é uma parte da palavra anterior
print('granola_tokens', tokenizer.convert_ids_to_tokens(granola_ids))

granola_ids [101, 36960, 10107, 10102, 11121, 14367, 102]
tipos de granola_ids <class 'list'>
granola_tokens ['[CLS]', 'barra', '##s', 'de', 'gran', '##ola', '[SEP]']


### *Tokens* especiais

Você deve ter notado os *tokens* especiais [CLS] e [SEP]. Esses *tokens* são adicionados automaticamente pelo método `.encode()`, então não precisamos nos preocupar com eles. O primeiro é um *token* de classificação que foi pré-treinado, utilizado nas tarefas de classificação. Desta forma, ao invés de fazer a média de todos os *tokens* e usá-los como uma representação de frase, é recomendado apenas pegar a saída do [CLS] que representa a frase inteira. [SEP], por sua vez, é inserido como um separador entre várias instâncias, usado por exemplo na predição da próxima sentença, separando uma frase da outra.

### *Tensor*

Como vimos acima, o tipo de dados dos IDs de cada *token* é uma lista de inteiros. Neste *notebook* vamos usar a biblioteca `transformers` em combinação com [*PyTorch*](https://pytorch.org/), que trabalha com tensores. Um tensor é um tipo especial de lista otimizada, normalmente usado em *deep learning*. Para converter nossos IDs dos *tokens* em um tensor, podemos simplesmente chamar um construtor de tensor passando a lista. Aqui, vamos usar um `LongTensor` que é usado para inteiros (para números de ponto flutuante,  usaríamos um `FloatTensor` ou apenas` Tensor`). 

O método `.encode ()` do tokenizer pode retornar um tensor em vez de uma lista, passando o parâmetro `return_tensors = 'pt'`, mas para fins de ilustração, faremos a conversão de uma lista para um tensor manualmente.

In [22]:
# Converte a lista de IDs para um tensor de IDs 
granola_ids = torch.LongTensor(granola_ids)
# Imprime os IDs
print('granola_ids', granola_ids)
print('tipos de granola_ids', type(granola_ids))

granola_ids tensor([  101, 36960, 10107, 10102, 11121, 14367,   102])
tipos de granola_ids <class 'torch.Tensor'>


## O modelo
Agora que pré-processamos nosso texto de entrada em um tensor de IDs (lembrando que cada valor de ID corresponde ao ID do *token* no vocabulário criado pelo tokenizador), podemos alimentar o modelo. O modelo sabe qual palavra está sendo processada porque ele sabe qual *token* pertence a determinado ID. 

No BERT, assim como na maioria dos modelos de linguagem baseados em [Transformer](https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf), a primeira camada é uma camada de *embedding*, cada *token* possui um *embedding* relacionado. No BERT, o *embedding* de um *token* é a soma de três tipos de *embeddings*: o *embedding* do *token* (gerado para o próprio *token*), o *embedding* do segmento (indica se o segmento faz parte da primeira ou da segunda sentença, não usado na inferência de uma única sentença) e o *embedding* de posição (distingue a posição do *token* na sentença). 

Para mais detalhes, veja [esse artigo](https://medium.com/@_init_/why-bert-has-3-embedding-layers-and-their-implementation-details-9c261108e28a). 

Abaixo, uma imagem do BERT retirada do artigo publicado.

![BERT embeddings visualization](https://github.com/HAILab-PUCPR/introducao-bert/blob/main/imagens/bert-embeddings.png?raw=true)

Para mais explicações sobre o modelo BERT, acesse [Jay Alammar's homepage](http://jalammar.github.io/).

### Inicializando o modelo

Para começar, primeiro precisamos inicializar o modelo. Assim como o tokenizer, o modelo é pré-treinado, o que nos permite usar um modelo de linguagem já pré-treinado para obter representações de *token* ou de senteças.

Devemos usar o mesmo modelo pré-treinado que o tokenizer usa (neste caso, `bert-base-multilingual-uncased`). Este é o modelo multilíngue BERT, treinado em texto em letras minúsculas. Desta forma, o tokenizer coloca o texto automaticamente em minúsculas para nós. A escolha do modelo, se deve ser caseado ou não, depende da tarefa. Tarefa como NER, por exemplo, podem requerer modelos treinados com maiúsculas e minúsculas (neste caso, troque "*uncased*" por "*cased*").

No exemplo abaixo, um argumento adicional foi fornecido para a inicialização do modelo. `output_hidden_states` fornece mais informações de saída. Por padrão, um `BertModel` irá retornar uma tupla, mas o conteúdo dessa tupla é diferente dependendo da configuração do modelo. Ao passar `output_hidden_states = True`, a tupla irá conter:

1. O último estado oculto `(batch_size, sequence_length, hidden_size)`
2. *pooler_output* do *token* de classificação `(batch_size, hidden_size)`
3. os estados_ocultos das saídas do modelo em cada camada e as saídas dos *embeddings* iniciais
   `(batch_size, sequence_length, hidden_size)`

### GPU x CPU

As placas gráficas (GPUs) são muito melhores em fazer operações em tensores do que uma CPU, portanto, sempre que disponível, executaremos os cálculos em GPU, como a CUDA (para isso, precisaremos de uma versão *torch* compatível com GPU.) 

Assim, movemos nosso modelo para o dispositivo correto: se estiver disponível, moveremos o modelo `.to ()` à GPU, caso contrário, permanecerá na CPU. É importante lembrar que o modelo e os dados a serem processados precisam estar no mesmo dispositivo. 

Finalmente, definimos o modelo para o modo de avaliação (`.eval`), em contraste com o modo de treinamento (` .train () `). Na avaliação, não temos por exemplo o *dropout*. 

In [23]:
model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states=True)

# Seta o dispositivo para GPU (cuda) se disponível, senão usa CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

## Inferência

O modelo foi inicializado e a string de entrada ("granola_ids") foi convertida em um tensor. Os modelos de linguagem (como
`BertModel`, usado acima) possuem um método` forward () `, chamado automaticamente ao chamar o objeto. Esse método envia o tensor de entrada para frente no modelo e retorna a saída. 

Como aqui trata-se de inferência, e não do treinamento ou ajuste (*fine-tuning*) do modelo, esta é a única etapa em que chamamos o modelo esperando uma saída (*output*). Portanto, não precisamos otimizar o modelo como calcular gradientes e fazer o  *backpropagation*. 

Definimos `torch.no_grad ()` na inferência para informar ao modelo que não faremos nenhum cálculo de gradiente e/ou retropropagação, tornando a inferência mais rápida e mais eficiente em termos de memória. 

Geralmente os métodos `model.eval ()` (veja acima) e `torch.no_grad ()` são usados juntos para avaliação e teste do modelo. Para treinar o modelo usamos o método `model.train ()` e o método `torch.no_grad ()` **não** deve ser usado.

### Lote (*batch*)

Abaixo, veremos um método chamado `.unsqueeze ()`, que "descomprime" um tensor adicionando uma dimensão extra. Então, nosso tensor de granola de tamanho `(7,)` irá se transformar em um tensor de `(1, 7)`, onde `1` é a dimensão da frase. Essas duas dimensões são requeridas pelo modelo: ele é otimizado para treinar em **lotes** (*batches*), como veremos adiante.

Um lote consiste em vários textos de entrada "ao mesmo tempo" (geralmente em potência de dois, por exemplo, 64). Com um tamanho de lote de 64 (ou seja, 64 frases de uma vez), o tamanho do lote seria `(64, n)` onde `64` é o número de frases e ` n` o
comprimento da sequência. Aqui, onde usamos apenas uma entrada, isso não é importante, mas ao ajustar um modelo, precisamos trabalhar com lotes, pois o cálculo do gradiente será melhor para grandes lotes. 

Nesses casos, `n` precisa ser o mesmo para todas as entradas, ou seja, não é possível ter uma sequência de 7 itens e uma de
12 itens (para lidar com isso, usamos técnicas de *padding*). O tamanho de entrada do modelo precisa ser `(n_input_sentences, seq_len)` onde `seq_len` pode ser determinado de diferentes maneiras.

Duas escolhas populares são: usar o texto mais longo do lote como `seq_len` (por exemplo, 12) e preencher textos mais curtos até
este comprimento, ou definir um comprimento de sequência máximo fixo para o modelo (normalmente 512) e preencher todos os itens até este comprimento. A última abordagem é mais fácil de implementar, mas não é eficiente em termos de memória e é computacionalmente mais pesada. Fica a seu critério.

In [24]:
print(granola_ids.size())
# descomprimir IDs para obter o tamanho do lote = 1 como dimensão extra
granola_ids = granola_ids.unsqueeze(0)
print(granola_ids.size())

print(type(granola_ids))
with torch.no_grad():
    out = model(input_ids=granola_ids)

# a saída é uma tupla
print(type(out))
# a tupla contém três elementos, que serão explicados abaixo
print(len(out))
# aqui serão listados apenas os estados ocultos do modelo (hidden_states)
hidden_states = out[2]
##print(len(hidden_states))

torch.Size([7])
torch.Size([1, 7])
<class 'torch.Tensor'>
<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>
3


### Estado oculto (*hidden state*)

Como visto acima, nós enviamos os IDs de nossos *tokens* de entrada por meio do método `model ()`, que chama internamente o
método `forward ()`. O `out` é uma tupla com todos os itens de saída relevantes, sendo o terceiro o mais importante, pois contém todos os estados_ocultos (`hidden_states`) do modelo após a execução de um *forward*. 

`hidden_states` é uma tupla da saída de cada camada no modelo para cada *token*. Na execução anterior, vimos que a tupla contém 13 itens. Quando você executa `print(model)` (célula abaixo), a arquitetura do BertModel é exibida (todas as camadas, de cima para baixo). O `hidden_states` inclui a saída da camada `embeddings` e a saída de todos os 12 `BertLayer` no codificador. A saída de cada camada tem um tamanho de `(batch_size, sequence_length, 768)`.

Em nosso exemplo, isso é `(1, 7, 768)` porque temos apenas uma string de entrada (tamanho do lote = 1), e nossa string de entrada foi tokenizada em sete IDs (comprimento de sequência de 7). `768` é o número de dimensões ocultas.

Como podemos ver, há mais uma camada após o codificador, chamada `pooler`, que não faz parte dos `hidden_states`. Esta camada é usada para "agrupar" a saída do *token* de classificação. Sua saída é retornada no segundo item da tupla de saída `out`, conforme visto antes.

Para mais informações sobre a arquitetura do BERT, leia [o artigo](https://arxiv.org/abs/1810.04805) e acesse o conteúdo
[The Illustrated BERT](http://jalammar.github.io/illustrated-bert/).

In [25]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

### Incorporação de sentença (*sentence embeddings*)

Agora que temos todos os `hidden_states`, podemos utilizá-lo em algumas tarefas. Por exemplo, para recuperar uma incorporação de frases (*sentence embeddings*) calculando a média de todos os *tokens*. Ou seja, vamos reduzir o tamanho de `(1, 7, 768)` para
`(1, 768)` onde `1` é o tamanho do lote e` 768` é o número de dimensões ocultas. 

Há diversas maneiras de fazer uma abstração de frase de *tokens*, dependendo da tarefa de PLN. Aqui, estamos usando a média. Por enquanto, usaremos apenas a saída da última camada do codificador, isto é, `hidden_states [-1]`. É importante indicar que queremos pegar o `torch.mean`_sobre um determinado eixo_. Uma vez que o tamanho da saída das camadas é `(1, 7, 768)`, queremos fazer a média sobre os sete *tokens*, que estão na segunda dimensão (`dim = 1`).


In [26]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

tensor([-8.8275e-03,  7.1126e-02, -1.1254e-01,  2.5045e-01, -2.2547e-01,
         4.8724e-01, -5.5254e-02, -3.3521e-01, -4.5242e-01,  2.6642e-01,
        -5.4808e-01, -1.4807e-02, -1.1958e-02, -1.7070e-01,  3.4750e-01,
        -1.7532e-01,  5.3843e-01, -1.8029e-01,  1.2748e-01,  3.0856e-02,
        -2.2213e-01, -1.5565e-01, -1.9474e-02, -1.0924e-01,  4.0263e-02,
        -2.2810e-02,  4.4041e-01,  2.1338e-01, -1.6954e-02,  8.1423e-02,
         2.6963e-01, -8.9791e-02,  1.3342e-01,  3.0469e-02,  7.9779e-02,
        -3.0330e-01, -6.7027e-02,  3.4837e-01,  1.7025e-01, -2.5633e-01,
        -2.3061e-02,  2.7046e-01, -1.0953e-01,  3.9915e-01,  2.0829e-01,
         1.4845e-01, -1.2066e-01,  1.5884e-01,  2.6394e-01, -2.5287e-01,
        -3.7953e-02, -1.4850e-01, -2.3999e-01, -6.9923e-02,  3.0408e-01,
        -4.2778e-02, -2.4120e-01, -2.8455e-01, -1.8484e-01,  1.3302e-01,
         2.7918e-01, -3.4697e-02,  2.8486e-02, -2.2839e-01, -1.2637e-01,
         5.0465e-02,  1.9249e-01,  1.9632e-01, -2.5

**Agora temos um vetor de 768 recursos que representam nossa sentença de entrada.** Mas podemos fazer mais! O artigo do BERT discute como alcançar os melhores resultados concatenando a saída das últimas quatro camadas.

![Visualização de embeddings de BERT](https://raw.githubusercontent.com/HAILab-PUCPR/introducao-bert/main/imagens/bert-feature-extraction-contextualized-embeddings.png)

Em nosso exemplo, isso significa que precisamos pegar as últimas quatro camadas de `hidden_states`, concatená-los e gerar a média. Nós queremos concatenar no eixo das dimensões ocultas de `768`. Como consequência, nosso vetor de saída concatenado irá
ser do tamanho `(1, 7, 3072)` onde `3072 = 4 * 768`, ou seja, a concatenação de quatro camadas com uma dimensão oculta de 768. O
vetor concatenado é muito maior do que a saída de apenas uma camada, o que significa que contém muito mais recursos.

Para algumas tarefas, esses recursos `3072` podem tem um desempenho melhor do que ` 768`.

Tendo um vetor de forma `(1, 7, 3072)`, ainda precisamos obter a média sobre a dimensão do *token*, como fizemos antes, ficando com um vetor de recurso de tamanho `(3072,)`.

In [27]:
# obter as ultimas quatro camadas
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# juntas as camadas em uma tupla e concatenar com a ultima dimensão
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# pegar a média do vetor concatenado sobre a dimensão do token
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding)
print(cat_sentence_embedding.size())

torch.Size([1, 7, 3072])
tensor([-0.0088,  0.0711, -0.1125,  ..., -0.1473,  0.0085, -0.2370])
torch.Size([3072])


## Salvando e carregando resultados ##

É possível usar o vetor de recurso gerado em outro modelo ou tarefa, para isso basta salvar o tensor com `torch.save` e carregá-lo em outro script com` torch.load`, gerando arquivos na extensão `.pt` (*PyTorch*). Não é possível ler o arquivo salvo com um editor de texto (é um objeto especial que permite uma des(compressão) eficiente). 

Também é possível salvar os tensores em um formato legível, convertendo em numpy e use algo como `np.savetxt ('tensor.txt', your_tensor.numpy ())`, porém essa abordagem não é recomendada (é melhor usar o `torch.save` ou outra técnica de compressão).

Ao usar `.cpu ()`, dizemos ao *PyTorch* que queremos mover o tensor de saída de volta da GPU para a CPU. Isso não é obrigatório, mas é uma boa prática, ao fazer extração de recursos, mover os dados para a CPU. Desta forma, ao carregá-lo, ele é carregado como um tensor de CPU em vez de um tensor CUDA. Depois podemos mover novamente para a GPU, se necessário, mas usar a CPU por padrão é uma boa ideia (o tensor deve estar na CPU para podermos convertê-lo para `.numpy ()`).


In [28]:
# sava nossa representação de sentença 
torch.save(cat_sentence_embedding.cpu(), 'my_sent_embed.pt')

# faz o load
loaded_tensor = torch.load('my_sent_embed.pt')
print(loaded_tensor)
print(loaded_tensor.size())

# converte para numpy para usar por ex com sklearn
np_loaded_tensor = loaded_tensor.numpy()
print(np_loaded_tensor)
print(type(np_loaded_tensor))


tensor([-0.0088,  0.0711, -0.1125,  ..., -0.1473,  0.0085, -0.2370])
torch.Size([3072])
[-0.00882755  0.07112557 -0.11253648 ... -0.14732859  0.00853673
 -0.23699594]
<class 'numpy.ndarray'>


### Dica ###
Você também poderá usar um serviço para gerar e devolver os *embeddings*, usando [**bert-as-service**](https://github.com/hanxiao/bert-as-service).